In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [6]:
# Load training data from CSV
train_df = pd.read_csv('train_data.csv')

# Filter images based on training data
train_images = train_df['image_name'].tolist()
train_images_dir = r"\NeuralCrash\tom_jerry\tom_jerry"

# Separate test images based on the test_data.csv file
test_df = pd.read_csv('test_data.csv')
test_images = test_df['image_name'].tolist()


# Data generator for training
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_images_dir,
    x_col='image_name',
    y_col=['jerry', 'tom'],
    target_size=(128, 128),
    batch_size=16,
    class_mode='raw',  # Use 'raw' for multi-output classification
    subset='training'  # Use only training images
)



Found 4382 validated image filenames.


In [ ]:

# CNN model architecture
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(512, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(2, activation='sigmoid')  # Output layer with 2 neurons for jerry and tom
])

# Compile the model
model.compile(optimizer='adam',
               loss='binary_crossentropy', 
               metrics=['accuracy'])


In [94]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=15
)

Epoch 1/15
547/547 [==============================] - 15s 24ms/step - loss: 0.8144 - accuracy: 0.5773
Epoch 2/15
547/547 [==============================] - 13s 24ms/step - loss: 0.4927 - accuracy: 0.6916
Epoch 3/15
547/547 [==============================] - 13s 24ms/step - loss: 0.3830 - accuracy: 0.7474
Epoch 4/15
547/547 [==============================] - 13s 23ms/step - loss: 0.3153 - accuracy: 0.7709
Epoch 5/15
547/547 [==============================] - 12s 23ms/step - loss: 0.2432 - accuracy: 0.7949
Epoch 6/15
547/547 [==============================] - 12s 23ms/step - loss: 0.1868 - accuracy: 0.7995
Epoch 7/15
547/547 [==============================] - 13s 23ms/step - loss: 0.1550 - accuracy: 0.8002
Epoch 8/15
547/547 [==============================] - 13s 23ms/step - loss: 0.1260 - accuracy: 0.8013
Epoch 9/15
547/547 [==============================] - 13s 23ms/step - loss: 0.1112 - accuracy: 0.8160
Epoch 10/15
547/547 [==============================] - 13s 24ms/step - loss: 0.098

In [7]:
# Data generator for test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=train_images_dir,  # Use the same directory for test images
    x_col='image_name',
    y_col=None,
    target_size=(128, 128),
    batch_size=1,
    shuffle=False,
    class_mode=None,
    subset='training'  # Use only test images
)

Found 1096 validated image filenames.


In [98]:
# Make predictions on the test set
predictions = model.predict(test_generator)

# Convert predictions to binary based on threshold
binary_predictions = (predictions > 0.5).astype(int)
image_names = test_generator.filenames

# Prepare predictions in CSV format
results = pd.DataFrame({'image_name': image_names, 'jerry': binary_predictions[:, 0], 'tom': binary_predictions[:, 1]})
results.to_csv('predictions.csv', index=False)

# Prepare predictions in CSV format
results = pd.DataFrame({'image_name': image_names, 'jerry': predictions[:, 0], 'tom': predictions[:, 1]})
results.to_csv('predictions1.csv', index=False)

1096/1096 [==============================] - 4s 3ms/step
